# 🚀 YouTube Viral Clipper - Google Colab

Aplikasi ini akan mengubah video YouTube menjadi klip viral dengan AI.

**Fitur:** AI-Powered Clip Selection | Smart Face Tracking | Word-by-Word Captions

---


## 📦 Cell 1: Setup & Instalasi (Smart Check)

Jalankan cell ini sekali untuk setup semua yang diperlukan.


In [ ]:
import os
import sys
import shutil
from pathlib import Path
from getpass import getpass
from google.colab import files

print("🚀 YouTube Viral Clipper - Smart Setup")
print("=" * 50)

# 1. Install dependencies (dengan pengecekan)
print("\n📦 Checking dependencies...")
deps_ok = True
mediapipe_ok = False

try:
    import dotenv
    import pytubefix
    import moviepy
    import faster_whisper
    import torch
    import google.generativeai
    import cv2
    import numpy
    import PIL
    # Test mediapipe secara khusus
    import mediapipe as mp
    # Test apakah solutions tersedia (ini yang sering error)
    test = mp.solutions.face_detection
    mediapipe_ok = True
    print("✅ Dependencies sudah terinstall!")
except (ImportError, AttributeError) as e:
    deps_ok = False
    print("⏳ Installing dependencies (ini mungkin memakan waktu beberapa menit)...")
    !pip install -q python-dotenv pytubefix moviepy faster-whisper torch google-generativeai opencv-python numpy pillow yt-dlp
    !apt-get update -qq && apt-get install -y -qq ffmpeg > /dev/null
    print("✅ Dependencies installed!")
    
    # Install mediapipe dengan protobuf yang kompatibel
    print("📦 Installing MediaPipe untuk face tracking...")
    print("   (Kode sudah diupdate untuk kompatibel dengan MediaPipe versi baru)")
    
    # Hapus mediapipe dari cache dan uninstall dengan benar
    modules_to_remove = [k for k in sys.modules.keys() if 'mediapipe' in k.lower()]
    for mod in modules_to_remove:
        del sys.modules[mod]
    
    # Uninstall mediapipe dan protobuf dengan benar
    print("   🧹 Cleaning up existing installations...")
    !pip uninstall -y mediapipe protobuf 2>/dev/null || true
    
    # Install protobuf 4.25.3 (kompatibel dengan mediapipe)
    # Warning tentang conflict dengan package lain adalah normal dan tidak akan menghentikan eksekusi
    print("   📦 Installing protobuf 4.25.3...")
    !pip install --no-cache-dir --force-reinstall "protobuf==4.25.3" 2>&1 | grep -v "WARNING\|ERROR" || true
    
    # Install mediapipe dengan cara yang lebih menyeluruh
    print("   📦 Installing mediapipe (ini mungkin memakan waktu)...")
    !pip uninstall -y mediapipe 2>/dev/null || true
    !pip install --no-cache-dir --force-reinstall mediapipe 2>&1 | grep -v "WARNING\|ERROR" || true
    
    # Tunggu sebentar untuk memastikan instalasi selesai
    import time
    time.sleep(2)
    
    # Hapus dari cache lagi setelah install
    modules_to_remove = [k for k in sys.modules.keys() if 'mediapipe' in k.lower() or 'protobuf' in k.lower()]
    for mod in modules_to_remove:
        del sys.modules[mod]
    
    # Verifikasi mediapipe setelah install dengan cara yang lebih robust
    print("   🧪 Testing MediaPipe installation...")
    try:
        # Test 1: Coba import dasar
        import mediapipe as mp
        print("      ✅ MediaPipe module imported")
        
        # Test 2: Coba cara baru (MediaPipe >= 0.10.x)
        try:
            from mediapipe.python.solutions.face_detection import FaceDetection
            test_detection = FaceDetection(model_selection=0, min_detection_confidence=0.5)
            test_detection.close()
            mediapipe_ok = True
            print("✅ MediaPipe verified (versi baru) - siap untuk face tracking!")
        except (ImportError, AttributeError) as e1:
            print(f"      ⚠️  Cara baru tidak bekerja: {e1}")
            # Test 3: Fallback ke cara lama (MediaPipe < 0.10.x)
            try:
                test = mp.solutions.face_detection
                mediapipe_ok = True
                print("✅ MediaPipe verified (versi lama) - siap untuk face tracking!")
            except AttributeError as e2:
                print(f"      ⚠️  Cara lama juga tidak bekerja: {e2}")
                # Test 4: Cek struktur mediapipe
                print(f"      🔍 MediaPipe attributes: {dir(mp)[:10]}...")
                raise AttributeError(f"MediaPipe tidak punya 'solutions' atau 'python.solutions': {e2}")
    except Exception as e:
        print(f"\n❌ MediaPipe installation failed: {e}")
        print("\n⚠️  SOLUSI ALTERNATIF:")
        print("   1. RESTART RUNTIME: Runtime → Restart runtime")
        print("   2. Setelah restart, jalankan Cell 1 LAGI")
        print("   3. Jika masih error, coba install manual di cell terpisah:")
        print("      !pip uninstall -y mediapipe protobuf")
        print("      !pip install protobuf==4.25.3")
        print("      !pip install mediapipe")
        print("   4. Setelah itu, restart runtime dan jalankan Cell 1 lagi")
        raise RuntimeError("MediaPipe installation failed. Silakan coba solusi alternatif di atas.")

if not mediapipe_ok:
    print("\n❌ MediaPipe tidak tersedia. Face tracking TIDAK bisa digunakan.")
    print("\n⚠️  TINDAKAN YANG DIPERLUKAN:")
    print("   1. RESTART RUNTIME: Runtime → Restart runtime")
    print("   2. Setelah restart, jalankan Cell 1 LAGI")
    print("   3. MediaPipe akan terinstall dengan benar setelah restart")
    print("\n💡 Setelah restart runtime, semua dependency akan terinstall ulang")
    print("   dan mediapipe akan bekerja untuk face tracking.")
    raise RuntimeError("MediaPipe diperlukan untuk face tracking. Silakan restart runtime dan jalankan Cell 1 lagi.")

# 2. Buat struktur direktori
print("\n📁 Creating directory structure...")
for dir_name in ['services', 'styles', 'utils', 'clips', 'temp']:
    os.makedirs(dir_name, exist_ok=True)
Path('services/__init__.py').touch()
Path('styles/__init__.py').touch()
Path('utils/__init__.py').touch()
print("✅ Directories created!")

# 3. Setup kode aplikasi
print("\n📂 Setting up application code...")
required_files = {
    'root': ['config.py', 'main.py'],
    'services': ['video_processor.py', 'youtube_downloader.py', 'whisper_transcriber.py', 
                 'gemini_selector.py', 'face_tracker.py', 'caption_maker.py'],
    'styles': ['caption_styles.py'],
    'utils': ['helpers.py']
}

# Cek file yang sudah ada
missing_files = []
for category, file_list in required_files.items():
    for file in file_list:
        if category == 'root':
            filepath = file
        else:
            filepath = f"{category}/{file}"
        if not os.path.exists(filepath):
            missing_files.append(filepath)

if missing_files:
    print(f"⚠️  {len(missing_files)} file(s) belum ada. Pilih metode setup:")
    print("   1. Clone dari GitHub (jika kode sudah di GitHub)")
    print("   2. Upload file manual")
    
    method = input("\nPilih metode (1 atau 2) [2]: ").strip() or "2"
    
    if method == "1":
        # Default repository URL (bisa diubah jika perlu)
        default_repo = "https://github.com/portdigital/testing-AI-Clipping-Software.git"
        repo_url = input(f"Masukkan URL GitHub repository [{default_repo}]: ").strip() or default_repo
        if repo_url:
            import subprocess
            import glob
            try:
                subprocess.run(['git', 'clone', repo_url, 'temp_repo'], check=True, 
                             stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
                # Pindahkan file ke root
                for file in glob.glob("temp_repo/*.py"):
                    shutil.move(file, os.path.basename(file))
                for folder in ['services', 'styles', 'utils']:
                    if os.path.exists(f"temp_repo/{folder}"):
                        for file in glob.glob(f"temp_repo/{folder}/*.py"):
                            shutil.move(file, f"{folder}/{os.path.basename(file)}")
                shutil.rmtree("temp_repo", ignore_errors=True)
                print("✅ Code cloned from GitHub!")
            except (subprocess.CalledProcessError, Exception) as e:
                print(f"❌ Error cloning repository: {e}")
                print("🔄 Fallback ke metode upload manual...")
                method = "2"  # Fallback ke upload manual
    
    if method == "2":
        print("\n📤 Upload file-file berikut (pilih semua sekaligus dengan Ctrl/Cmd+Click):")
        print("   - config.py, main.py")
        print("   - services/*.py (semua file)")
        print("   - styles/*.py (semua file)")
        print("   - utils/*.py (semua file)")
        uploaded = files.upload()
        
        # Pindahkan file ke lokasi yang benar
        for filename in uploaded.keys():
            if '/' in filename and os.path.exists(filename):
                pass  # Sudah di lokasi benar
            elif filename.startswith(('services/', 'styles/', 'utils/')):
                target_dir = os.path.dirname(filename)
                os.makedirs(target_dir, exist_ok=True)
                if not os.path.exists(filename):
                    base_name = os.path.basename(filename)
                    if os.path.exists(base_name):
                        shutil.move(base_name, filename)
            # File root seperti config.py, main.py tetap di root
        print("✅ Files uploaded!")
    
    # Verifikasi lagi
    still_missing = []
    for category, file_list in required_files.items():
        for file in file_list:
            if category == 'root':
                filepath = file
            else:
                filepath = f"{category}/{file}"
            if not os.path.exists(filepath):
                still_missing.append(filepath)
    
    if still_missing:
        print(f"\n❌ Masih ada {len(still_missing)} file yang missing:")
        for f in still_missing:
            print(f"   - {f}")
        raise FileNotFoundError("Silakan upload file yang missing dan jalankan cell ini lagi.")
else:
    print("✅ Semua file sudah ada!")

# 4. Setup API Key
print("\n🔑 Setting up API Key...")
if os.path.exists('.env'):
    print("✅ .env file sudah ada!")
    with open('.env', 'r') as f:
        if 'GEMINI_API_KEY=' in f.read():
            print("✅ API Key sudah dikonfigurasi!")
        else:
            api_key = getpass("Masukkan Gemini API Key: ")
            with open('.env', 'a') as f:
                f.write(f'\nGEMINI_API_KEY={api_key}\n')
            print("✅ API Key configured!")
else:
    api_key = getpass("Masukkan Gemini API Key (dapatkan di https://makersuite.google.com/app/apikey): ")
    with open('.env', 'w') as f:
        f.write(f'GEMINI_API_KEY={api_key}\n')
        f.write('OUTPUT_DIR=./clips\n')
        f.write('TEMP_DIR=./temp\n')
        f.write('WHISPER_MODEL=medium\n')
    print("✅ API Key configured!")

# 5. Test import
print("\n🧪 Testing imports...")
sys.path.insert(0, os.getcwd())
try:
    from services.video_processor import VideoProcessor
    from styles.caption_styles import CAPTION_STYLES
    print("✅ All modules imported successfully!")
except Exception as e:
    print(f"❌ Import error: {e}")
    print("💡 Pastikan semua file sudah terupload dengan benar.")
    raise

print("\n" + "=" * 50)
print("✅ Setup selesai! Lanjutkan ke Cell 2 untuk generate video.")
print("=" * 50)


## 🎬 Cell 2: Generate Video

Jalankan cell ini untuk membuat klip viral dari video YouTube.


In [ ]:
import sys
import os
from pathlib import Path
from google.colab import files
import zipfile

# Pastikan setup sudah dilakukan
if not os.path.exists('services/video_processor.py'):
    raise FileNotFoundError("❌ Setup belum dilakukan! Jalankan Cell 1 terlebih dahulu.")

# Import modules
sys.path.insert(0, os.getcwd())
from services.video_processor import VideoProcessor
from styles.caption_styles import CAPTION_STYLES

print("🎬 YouTube Viral Clipper - Generate Video")
print("=" * 50)

# Input parameter
youtube_url = input("\n🔗 Masukkan YouTube URL: ").strip()
if not youtube_url:
    raise ValueError("URL tidak boleh kosong!")

num_clips = int(input("📊 Jumlah klip [3]: ") or "3")
max_duration = int(input("⏱️  Durasi maksimal per klip (detik) [60]: ") or "60")
min_duration = int(input("⏱️  Durasi minimal per klip (detik) [20]: ") or "20")

# Tampilkan caption styles
print("\n🎨 Available Caption Styles:")
style_keys = list(CAPTION_STYLES.keys())
for i, key in enumerate(style_keys, 1):
    print(f"   {i}. {CAPTION_STYLES[key]['name']}")

style_choice = int(input(f"\n🎨 Pilih style (1-{len(style_keys)}) [1]: ") or "1")
selected_style = style_keys[style_choice - 1] if 1 <= style_choice <= len(style_keys) else 'clean_white'

# Proses video
print("\n" + "=" * 50)
print("🚀 Starting video processing...")
print("=" * 50)

processor = VideoProcessor(caption_style=selected_style)
outputs, title = processor.process_video(youtube_url, num_clips, min_duration, max_duration)

print("\n" + "=" * 50)
print("✅ Processing complete!")
print("=" * 50)
print(f"📹 Source: {title}")
print(f"📁 Location: ./clips/")
print(f"📊 Generated {len(outputs)} clip(s)")

# Download hasil
print("\n📥 Preparing download...")
clips_dir = Path('./clips')
if clips_dir.exists() and any(clips_dir.iterdir()):
    zip_path = 'viral_clips.zip'
    with zipfile.ZipFile(zip_path, 'w') as zipf:
        for clip_file in clips_dir.glob('*.mp4'):
            zipf.write(clip_file, clip_file.name)
    
    files.download(zip_path)
    print("✅ Download started! File akan terdownload otomatis.")
else:
    print("⚠️  No clips found in ./clips directory")
